In [1]:
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
# display all columns
pd.options.display.max_columns = None

In [2]:
# Utils
from sklearn import preprocessing
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None, test: bool = False, test_dataset = None):
    if test == True:
        if mean is None:
            mean = df[name].mean()
        if sd is None:
            sd = df[name].std()
        test_dataset[name] = (test_dataset[name] - mean) / sd
    else:
        if mean is None:
            mean = df[name].mean()
        if sd is None:
            sd = df[name].std()
        df[name] = (df[name] - mean) / sd

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

In [3]:
path = "/Users/nguyenquangminh/Desktop/Thesis/Dataset/CICDataset/CIC-IDS-2017/TrafficLabelling /Wednesday-workingHours.pcap_ISCX.csv"
# custom_dataset_path = "../../../Dataset/wrk_benchmark.pcap_Flow.csv"
custom_dataset_path = "../../../../PROJECT/solo-projects/IDS-Dataset/capture.pcap_Flow.csv"
dataset = pd.read_csv(filepath_or_buffer=path)
dataset.drop(columns=[' Fwd Header Length.1'], inplace=True) # duplicate
custom_dataset = pd.read_csv(filepath_or_buffer=custom_dataset_path)
dataset[" Label"].replace("DoS Hulk", "Malicious", inplace=True)
dataset[" Label"].replace("DoS GoldenEye", "Malicious", inplace=True)
dataset[" Label"].replace("DoS slowloris", "Malicious", inplace=True)
dataset[" Label"].replace("DoS Slowhttptest", "Malicious", inplace=True)
dataset[" Label"].replace("Heartbleed", "Malicious", inplace=True)
print(dataset[" Label"].value_counts())

BENIGN       440031
Malicious    252672
Name:  Label, dtype: int64


In [4]:
spoof_dataset = custom_dataset[custom_dataset["Dst IP"] == "58.186.71.236"].copy(deep=True)
spoof_dataset.columns = dataset.columns
spoof_dataset

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label


In [5]:

custom_dataset.columns = dataset.columns
custom_dataset[" Label"] = "BENIGN"

dataset.drop(columns=['Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Timestamp'], inplace=True)
custom_dataset.drop(columns=['Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Timestamp'], inplace=True)

In [6]:
for column in custom_dataset.columns:
    if column != " Label":
        encode_numeric_zscore(df=dataset, name=column, test=True, test_dataset=custom_dataset)
    else:
        print(encode_text_index(custom_dataset, column))
        
for column in dataset.columns:
    if column != " Label":
        encode_numeric_zscore(dataset, column)
    else:
        print(encode_text_index(dataset, column))

['BENIGN']
['BENIGN' 'Malicious']


In [7]:
dataset

,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,-0.356503,-0.642646,-0.653857,-0.011451,-0.009362,-0.089086,-0.007581,-0.376966,-0.176667,-0.346068,-0.366591,-0.633357,-0.430940,-0.684609,-0.599827,NaN,NaN,-0.440408,-0.582296,-0.595270,-0.050133,-0.648683,-0.459336,-0.563341,-0.593147,-0.116252,-0.415678,-0.275916,-0.328068,-0.355621,-0.114315,-0.209906,NaN,NaN,NaN,-0.014196,-0.011643,-0.298339,-0.130224,-0.280643,-0.652576,-0.737535,-0.667906,-0.507522,-0.328313,-0.209906,-0.018383,-0.490125,1.179856,3.750547,NaN,-0.018422,0.776269,-0.745303,-0.346068,-0.684609,NaN,NaN,NaN,NaN,NaN,NaN,-0.011451,-0.089086,-0.009362,-0.007581,-0.425003,-0.072539,-0.008559,-1.069400,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
1,-0.336856,-0.642646,-0.654742,0.001932,-0.005298,-0.062153,-0.007438,-0.256055,-0.294156,-0.284940,-0.227513,-0.573294,-0.523839,-0.610372,-0.518520,NaN,NaN,-0.447248,-0.582294,-0.596266,-0.060562,-0.648671,-0.459331,-0.563338,-0.593144,-0.116252,-0.415666,-0.275905,-0.328058,-0.355611,-0.114314,-0.209906,NaN,NaN,NaN,0.008051,-0.003720,-0.226625,0.206534,-0.498652,-0.592980,-0.674429,-0.595750,-0.505694,-0.328313,-0.209906,-0.018383,2.040292,-0.847560,-0.266627,NaN,-0.018422,-0.975862,-0.689719,-0.284940,-0.610372,NaN,NaN,NaN,NaN,NaN,NaN,0.001932,-0.062153,-0.005298,-0.007438,2.013552,-0.166315,-0.002966,0.828623,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
2,-0.355994,-0.642646,-0.654727,0.000594,-0.004282,0.421001,-0.006178,2.221784,-0.294156,1.614047,2.430793,-0.033110,-0.523839,-0.033784,0.140878,NaN,NaN,-0.447241,-0.582279,-0.596247,-0.060562,-0.648657,-0.459325,-0.563322,-0.593123,-0.116252,-0.415648,-0.275895,-0.328036,-0.355590,-0.114314,-0.209906,NaN,NaN,NaN,0.006005,-0.002095,-0.269869,0.046149,-0.498652,-0.056989,0.250160,0.189530,-0.249501,-0.328313,-0.209906,-0.018383,2.040292,-0.847560,-0.266627,NaN,-0.018422,-0.975862,0.221294,1.614047,-0.033784,NaN,NaN,NaN,NaN,NaN,NaN,0.000594,0.421001,-0.004282,-0.006178,2.013552,0.082616,-0.004364,0.828623,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
3,-0.336856,-0.642646,-0.654397,0.009962,0.001815,0.469998,-0.004612,1.787831,-0.294156,0.903958,1.516334,0.538445,-0.523839,0.003836,0.290375,NaN,NaN,-0.447157,-0.582082,-0.595919,-0.060562,-0.648327,-0.459250,-0.563133,-0.592799,-0.116252,-0.415224,-0.275772,-0.327679,-0.355087,-0.114314,-0.209906,NaN,NaN,NaN,0.020325,0.007047,-0.294925,-0.105545,-0.498652,0.510129,0.159348,0.231527,-0.223607,-0.328313,-0.209906,-0.018383,2.040292,-0.847560,-0.266627,NaN,-0.018422,-0.975862,0.108090,0.903958,0.003836,NaN,NaN,NaN,NaN,NaN,NaN,0.009962,0.469998,0.001815,-0.004612,2.013552,-0.201858,0.005424,0.82

In [8]:
dataset[" Label"].value_counts()

0    440031
1    252672
Name:  Label, dtype: int64

In [9]:
custom_dataset

,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3.382698,-0.642646,-0.654752,-0.011451,-0.009362,-0.090059,-0.007584,-0.386904,-0.294156,-0.384128,-0.366591,-0.635652,-0.523839,-0.692133,-0.599827,NaN,NaN,-0.447249,-0.582296,-0.596267,-0.060554,-0.648683,-0.459336,-0.563341,-0.593147,-0.116252,-0.415678,-0.275916,-0.328068,-0.355621,-0.114315,-0.209906,NaN,NaN,NaN,-0.013940,-0.011643,-0.197570,0.912231,-0.498652,-0.654854,-0.753789,-0.667906,-0.507522,-0.328313,4.764021,54.399126,-0.490125,1.179856,-0.266627,NaN,-0.018422,0.776269,-0.767914,-0.384128,-0.692133,NaN,NaN,NaN,NaN,NaN,NaN,-0.012789,-0.090059,-0.010378,-0.007584,5.074698,-0.201858,-0.008559,-0.436726,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
1,-0.333422,-0.642646,-0.649835,-0.008774,-0.008346,-0.090059,-0.007584,-0.386904,-0.294156,-0.384128,-0.366591,-0.635652,-0.523839,-0.692133,-0.599827,NaN,NaN,-0.437859,-0.573362,-0.590796,-0.060562,-0.643769,-0.449815,-0.554057,-0.587680,-0.116252,-0.409374,-0.254000,-0.328068,-0.347585,-0.088384,-0.209906,NaN,NaN,NaN,-0.008570,-0.009002,-0.298376,-0.130761,-0.498652,-0.654854,-0.753789,-0.667906,-0.507522,6.420054,9.737948,-0.018383,-0.490125,7.262104,-0.266627,NaN,-0.018422,-0.975862,-0.767914,-0.384128,-0.692133,NaN,NaN,NaN,NaN,NaN,NaN,-0.012789,-0.090059,-0.010378,-0.007584,5.074698,-0.132687,-0.008559,0.828623,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
2,0.276468,-0.642646,-0.654752,-0.011451,-0.009362,-0.090059,-0.007584,-0.386904,-0.294156,-0.384128,-0.366591,-0.635652,-0.523839,-0.692133,-0.599827,NaN,NaN,-0.447251,-0.582296,-0.596267,-0.060558,-0.648683,-0.459336,-0.563341,-0.593147,-0.116252,-0.415678,-0.275916,-0.328068,-0.355621,-0.114315,-0.209906,NaN,NaN,NaN,-0.013940,-0.011643,-0.089997,2.025084,-0.498652,-0.654854,-0.753789,-0.667906,-0.507522,-0.328313,4.764021,54.399126,-0.490125,1.179856,-0.266627,NaN,-0.018422,0.776269,-0.767914,-0.384128,-0.692133,NaN,NaN,NaN,NaN,NaN,NaN,-0.012789,-0.090059,-0.010378,-0.007584,-0.360216,-0.201858,-0.008559,-0.436726,-0.131646,-0.100396,-0.14867,-0.104366,-0.579979,-0.105686,-0.58524,-0.57078,0
3,0.634887,-0.642646,-0.654752,-0.011451,-0.009362,-0.090059,-0.007584,-0.386904,-0.294156,-0.384128,-0.366591,-0.635652,-0.523839,-0.692133,-0.599827,NaN,NaN,-0.447249,-0.582296,-0.596267,-0.060555,-0.648683,-0.459336,-0.563341,-0.593147,-0.116252,-0.415678,-0.275916,-0.328068,-0.355621,-0.114315,-0.209906,NaN,NaN,NaN,-0.013940,-0.011643,-0.182630,1.066794,-0.498652,-0.654854,-0.753789,-0.667906,-0.507522,-0.328313,4.764021,54.399126,-0.490125,1.179856,-0.266627,NaN,-0.018422,0.776269,-0.767914,-0.384128,-0.692133,NaN,NaN,NaN,NaN,NaN,NaN,-0.012789,-0.090059,-0.010378,-0.007584,5.07

In [10]:
custom_dataset[" Label"].value_counts()

0    2529
Name:  Label, dtype: int64

In [11]:
nan_columns = dataset.columns[dataset.isna().any()]
print(nan_columns)
test_nan_columns = custom_dataset.columns[custom_dataset.isna().any()]
print(test_nan_columns)
dataset.drop(columns=nan_columns, inplace=True)
custom_dataset.drop(columns=nan_columns, inplace=True)

Index(['Flow Bytes/s', ' Flow Packets/s', ' Bwd PSH Flags', ' Fwd URG Flags',
       ' Bwd URG Flags', ' CWE Flag Count', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')
Index(['Flow Bytes/s', ' Flow Packets/s', ' Bwd PSH Flags', ' Fwd URG Flags',
       ' Bwd URG Flags', ' CWE Flag Count', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')


In [12]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

# Break into X (predictors) & y (prediction)
x, y = to_xy(dataset,' Label')

# Create a test/train split.  25% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
16236/16236 - 10s - loss: 0.0452 - val_loss: 0.0295 - 10s/epoch - 646us/step
Epoch 2/1000
16236/16236 - 12s - loss: 0.0273 - val_loss: 0.0232 - 12s/epoch - 709us/step
Epoch 3/1000
16236/16236 - 11s - loss: 0.0252 - val_loss: 0.0231 - 11s/epoch - 694us/step
Epoch 4/1000
16236/16236 - 10s - loss: 0.0236 - val_loss: 0.0215 - 10s/epoch - 622us/step
Epoch 5/1000
16236/16236 - 10s - loss: 0.0226 - val_loss: 0.0325 - 10s/epoch - 615us/step
Epoch 6/1000
16236/16236 - 10s - loss: 0.0218 - val_loss: 0.0203 - 10s/epoch - 614us/step
Epoch 7/1000
16236/16236 - 10s - loss: 0.0200 - val_loss: 0.0185 - 10s/epoch - 624us/step
Epoch 8/1000
16236/16236 - 9s - loss: 0.0195 - val_loss: 0.0152 - 9s/epoch - 581us/step
Epoch 9/1000
16236/16236 - 9s - loss: 0.0193 - val_loss: 0.0153 - 9s/epoch - 580us/step
Epoch 10/1000
16236/16236 - 9s - loss: 0.0183 - val_loss: 0.0182 - 9s/epoch - 578us/step
Epoch 11/1000
16236/16236 - 9s - loss: 0.0177 - val_loss: 0.0133 - 9s/epoch - 581us/step
Epoch 12/1000
16

In [13]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

5412/5412 [==============================] - 2s 400us/step
Validation score: 0.996483346422137


In [14]:
x_custom, y_custom = to_xy(custom_dataset, " Label")

In [15]:
# Measure accuracy
pred = model.predict(x_custom)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_custom,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation sore: {}".format(score))



80/80 [==============================] - 0s 342us/step
Validation sore: 0.9849742981415579


In [16]:
# 75 - 99%
spoof_dataset.drop(columns=['Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Timestamp'], inplace=True)
spoof_dataset.drop(columns=nan_columns, inplace=True)

for column in spoof_dataset.columns:
    if column != " Label":
        encode_numeric_zscore(df=dataset, name=column, test=True, test_dataset=spoof_dataset)
    else:
        print(encode_text_index(spoof_dataset, column))

[]


In [17]:
spoof_dataset

,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label


In [18]:
x_custom, y_custom = to_xy(spoof_dataset, " Label")
# Measure accuracy
pred = model.predict(x_custom)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_custom,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation sore: {}".format(score))

ValueError: Expected input data to be non-empty.

In [ ]:
# 71-100%